In [70]:
%matplotlib inline

In [71]:
import pandas as pd
import numpy as np

import json
import pickle
import re

import matplotlib.pyplot as plt

# Variables

Here is the selected variables:

- `TRIAL_INDEX`
- `EYE_USED`
- `CURRENT_FIX_X`
- `CURRENT_FIX_Y`
- `CURRENT_FIX_START`
- `CURRENT_FIX_DURATION`
- `NEXT_SAC_END_X`
- `NEXT_SAC_END_Y`
- `NEXT_SAC_AMPLITUDE`
- `NEXT_SAC_DIRECTION`
- `NEXT_SAC_DURATION`
- `NEXT_SAC_ANGLE`
- `NEXT_SAC_AVG_VELOCITY`

Since the Timestamp of the events is not at the right time, we need to synchronize Mouse data and eyes data. To do such, we sent a MSG containing `TRIAL_START=XXX-XX-XX XX:XX:XX` at the begining of the trial and `TRIAL_END=XXX-XX-XX XX:XX:XX` at the end.

# Load Data

In [72]:
part_id = 714

In [73]:
df = pd.read_csv("../data/part_{0}/part_{0}.csv".format(part_id), sep="\t")
df.head()

,TRIAL_INDEX,EYE_USED,CURRENT_FIX_X,CURRENT_FIX_Y,CURRENT_FIX_START,CURRENT_FIX_DURATION,NEXT_SAC_END_X,NEXT_SAC_END_Y,NEXT_SAC_AMPLITUDE,NEXT_SAC_DIRECTION,NEXT_SAC_DURATION,NEXT_SAC_ANGLE,NEXT_SAC_AVG_VELOCITY
0,1,RIGHT,"385,20","66,40",6,97,"778,60","570,10","17,55",DOWN,73,"-51,57","240,45"
1,1,RIGHT,"775,50","564,50",176,53,"882,60","497,00","3,75",RIGHT,114,"34,88","32,91"
2,1,RIGHT,"890,30","487,60",343,179,"798,10","547,40","3,16",LEFT,37,"-146,84","85,51"
3,1,RIGHT,"797,10","559,20",559,151,"814,50","656,30","2,76",DOWN,27,"-79,39","102,09"
4,1,RIGHT,"820,40","650,10",737,215,"847,70","196,70","12,64",UP,52,"86,35","243,03"


In [74]:
msg = pd.read_csv("../data/part_{0}/part_{0}_msg.csv".format(part_id), sep="\t")
msg.head()

,TRIAL_INDEX,CURRENT_MSG_TIME,CURRENT_MSG_TEXT
0,1,11,TRIAL START = 2018-04-03 17:21:14
1,1,59990,TRIAL END = 2018-04-03 17:22:14
2,2,6,TRIAL START = 2018-04-03 17:22:54
3,2,60290,TRIAL END = 2018-04-03 17:23:54
4,3,6,TRIAL START = 2018-04-03 17:24:26


In [75]:
mrs_json = json.load(open("../data/part_{0}/records-{0}.mrs".format(part_id)))

In [76]:
config = pickle.load(open("../data/part_{0}/part_{0}.cfg".format(part_id), 'rb'))

# Extract data

In [77]:
# Extract scroll data from json file
def extract_scroll(mrs_json, idx):
    # There is two key format: scroll|mouse-website_id-part_id or scroll|mouse-website_id
    # So we need to check that out
    r = re.compile("scroll-"+str(idx)+"(?!\d)")

    for item in mrs_json:
        match = list(filter(r.match, list(item.keys())))
        
        if(len(match) > 0):
            return pd.DataFrame(item[match[0]])
    
    return None

In [78]:
# Build scroll offset dataset
df_scroll = pd.DataFrame()
for i in range(1,19):
    scroll = None
    scroll = extract_scroll(mrs_json, i)
    scroll["website_id"] = i
    df_scroll = pd.concat([df_scroll, scroll])

In [79]:
# Extract right Datetime
# Timestamp is gave by `new Date().getTime()` in Javascript which is in ms
# And since this same function give UTC time, we need to add 1H


time_to_add = 0
# There was a time change on 25 March 2018, so the time shift between the datasets is not 1h anymore but 2h
if(part_id > 700 and part_id < 712):
    time_to_add = 1
else:
    time_to_add = 2

df_scroll["datetime"] = pd.to_datetime(df_scroll["timestamp"], unit="ms") + pd.Timedelta(hours=time_to_add)

In [80]:
df_scroll.head()

,offset,timestamp,website_id,datetime
0,1,1522769842319,1,2018-04-03 17:37:22.319
1,2,1522769842325,1,2018-04-03 17:37:22.325
2,4,1522769842333,1,2018-04-03 17:37:22.333
3,7,1522769842343,1,2018-04-03 17:37:22.343
4,13,1522769842355,1,2018-04-03 17:37:22.355


# Messages cleaning

In [81]:
# Delete useless msg
msg = msg[~msg["CURRENT_MSG_TEXT"].str.contains("!MODE RECORD")].reset_index(drop=True)
msg.head()

,TRIAL_INDEX,CURRENT_MSG_TIME,CURRENT_MSG_TEXT
0,1,11,TRIAL START = 2018-04-03 17:21:14
1,1,59990,TRIAL END = 2018-04-03 17:22:14
2,2,6,TRIAL START = 2018-04-03 17:22:54
3,2,60290,TRIAL END = 2018-04-03 17:23:54
4,3,6,TRIAL START = 2018-04-03 17:24:26


In [82]:
# Extract message datetime in a new column
def split_equal(row):
    string = row["CURRENT_MSG_TEXT"].split("=")
    row["EVENT_NAME"] = string[0].strip()
    row["EVENT_DATETIME"] = string[1].strip()
    
    del row["CURRENT_MSG_TEXT"]
    
    return row

msg = msg.apply(split_equal, axis=1)
msg.head()

,TRIAL_INDEX,CURRENT_MSG_TIME,EVENT_NAME,EVENT_DATETIME
0,1,11,TRIAL START,2018-04-03 17:21:14
1,1,59990,TRIAL END,2018-04-03 17:22:14
2,2,6,TRIAL START,2018-04-03 17:22:54
3,2,60290,TRIAL END,2018-04-03 17:23:54
4,3,6,TRIAL START,2018-04-03 17:24:26


In [83]:
msg["EVENT_DATETIME"] = pd.to_datetime(msg["EVENT_DATETIME"])

# Other cleaning

In [84]:
# Left, Right, Up or Down
df["NEXT_SAC_DIRECTION"] = df["NEXT_SAC_DIRECTION"].astype("category")
df["NEXT_SAC_DIRECTION"].cat.categories

Index(['.', 'DOWN', 'LEFT', 'RIGHT', 'UP'], dtype='object')

In [85]:
df["CURRENT_FIX_Y"] = pd.to_numeric(df["CURRENT_FIX_Y"].str.replace(',','.'))
df["CURRENT_FIX_X"] = pd.to_numeric(df["CURRENT_FIX_X"].str.replace(',','.'))


# Last fixations does not have NEXT_SAC information
df["NEXT_SAC_AMPLITUDE"] = pd.to_numeric(df["NEXT_SAC_AMPLITUDE"].str.replace(".", "").str.replace(",", "."))
df["NEXT_SAC_END_X"] = pd.to_numeric(df["NEXT_SAC_END_X"].str.replace(".", "").str.replace(",", "."))
df["NEXT_SAC_END_Y"] = pd.to_numeric(df["NEXT_SAC_END_Y"].str.replace(".", "").str.replace(",", "."))
df["NEXT_SAC_DURATION"] = pd.to_numeric(df["NEXT_SAC_DURATION"].str.replace(".", "").str.replace(",", "."))
df["NEXT_SAC_ANGLE"] = pd.to_numeric(df["NEXT_SAC_ANGLE"].str.replace(".", "").str.replace(",", "."))
df["NEXT_SAC_AVG_VELOCITY"] = pd.to_numeric(df["NEXT_SAC_AVG_VELOCITY"].str.replace(".", "").str.replace(",", "."))

# Time Sync

In [86]:
# Time sync
def sync_time(cell, msg):
    timedelta = cell - msg["CURRENT_MSG_TIME"][0]
    to_return = msg.loc[0, "EVENT_DATETIME"] + pd.Timedelta(milliseconds=timedelta)
    return to_return

In [87]:
def sync_and_clean(group):
    trial_index = group['TRIAL_INDEX'].unique()[0]
    
    msg_start_trial = msg.query("TRIAL_INDEX == @trial_index and EVENT_NAME == 'TRIAL START'").reset_index(drop=True)
    msg_end_trial = msg.query("TRIAL_INDEX == @trial_index and EVENT_NAME == 'TRIAL END'").reset_index(drop=True)

    group["DATETIME"] = group["CURRENT_FIX_START"].apply(lambda x: sync_time(x, msg_start_trial))
    
    group = group[group["DATETIME"] > msg_start_trial["EVENT_DATETIME"][0]]
    group = group[group["DATETIME"] < msg_end_trial["EVENT_DATETIME"][0]]
    
    return group

In [88]:
df = df.groupby("TRIAL_INDEX").apply(sync_and_clean).reset_index(drop = True)

# Websites and Conditions

In [89]:
# Condition 1:  Free    + NoPub
# Condition 2:  Target  + NoPub
# Condition 3:  Free    + Skin
# Condition 4:  Target  + Skin
# Condition 5:  Free    + Skin/MPU
# Condition 6:  Target  + Skin/MPU
def get_condition(trial_num):
    data = config["rand_weblist"][trial_num - 1]
    if(data["type"] == "free" and data["ad_id"] == 0 and data["mpu_id"] == 0):
        return 1
    elif(data["type"] == "target" and data["ad_id"] == 0 and data["mpu_id"] == 0):
        return 2
    elif(data["type"] == "free" and data["ad_id"] > 0 and data["mpu_id"] == 0):
        return 3
    elif(data["type"] == "target" and data["ad_id"] > 0 and data["mpu_id"] == 0):
        return 4
    elif(data["type"] == "free" and data["ad_id"] > 0 and data["mpu_id"] > 0):
        return 5
    elif(data["type"] == "target" and data["ad_id"] > 0 and data["mpu_id"] > 0):
        return 6
    
    return None

def condition_string(num):
    if(num == 1):
        return "Free + NoPub"
    elif(num == 2):
        return "Target + NoPub"
    elif(num == 3):
        return "Free + Skin"
    elif(num == 4):
        return "Target + Skin"
    elif(num == 5):
        return "Free + Skin/MPU"
    elif(num == 6):
        return "Target + Skin/MPU"
    else:
        return None

In [90]:
def get_website_id(trial_num):
    return config["rand_weblist"][trial_num - 1]["id"]

In [91]:
df["WEBSITE_ID"] = df["TRIAL_INDEX"].apply(get_website_id)

In [92]:
df["CONDITION"] = df["TRIAL_INDEX"].apply(get_condition)

# Offset Sync

In [93]:
def get_last_offset(trial_scroll, date_eye):
    result = trial_scroll[trial_scroll["datetime"] < date_eye]
    if(result.empty):
        return 0
    else:
        return result.iloc[-1]["offset"]

In [94]:
def get_offset(group):
    website_id = group['WEBSITE_ID'].unique()[0]
    group["OFFSET"] = group["DATETIME"].apply(lambda x: get_last_offset(df_scroll.query("website_id == "+str(website_id)), x))
    return group

In [95]:
df = df.groupby("TRIAL_INDEX").apply(get_offset)

In [96]:
df.groupby("TRIAL_INDEX")["OFFSET"].unique()

TRIAL_INDEX
1     [0, 100, 200, 209, 300, 400, 407, 500, 600, 70...
2     [0, 97, 100, 155, 200, 276, 300, 340, 400, 499...
3     [0, 93, 100, 166, 300, 498, 500, 800, 809, 120...
4     [0, 95, 100, 280, 400, 600, 619, 700, 900, 109...
5     [0, 55, 200, 300, 319, 400, 463, 500, 507, 600...
6     [0, 100, 300, 444, 500, 599, 600, 760, 800, 85...
7     [0, 32, 100, 132, 200, 204, 300, 498, 500, 528...
8     [0, 2, 200, 300, 317, 400, 408, 500, 700, 684,...
9     [0, 92, 100, 122, 200, 285, 300, 361, 400, 422...
10    [0, 98, 100, 119, 400, 600, 770, 800, 805, 100...
11    [0, 100, 187, 200, 242, 400, 430, 500, 522, 60...
12    [0, 80, 100, 200, 300, 353, 400, 494, 500, 600...
13    [0, 81, 100, 135, 300, 389, 400, 500, 593, 600...
14    [0, 100, 65, 19, 400, 488, 500, 600, 700, 800,...
15    [0, 1, 100, 226, 300, 600, 700, 800, 999, 1000...
16    [0, 86, 100, 146, 300, 500, 540, 700, 907, 100...
17    [0, 28, 200, 219, 302, 400, 500, 657, 700, 792...
18    [0, 99, 100, 200, 354, 400, 40

In [97]:
df["Y_OFFSET"] = df["CURRENT_FIX_Y"] + df["OFFSET"]

# Export

In [98]:
df.to_csv("../data/part_{0}/part_{0}_clean.csv".format(part_id), index=False)

In [99]:
df.head()

,TRIAL_INDEX,EYE_USED,CURRENT_FIX_X,CURRENT_FIX_Y,CURRENT_FIX_START,CURRENT_FIX_DURATION,NEXT_SAC_END_X,NEXT_SAC_END_Y,NEXT_SAC_AMPLITUDE,NEXT_SAC_DIRECTION,NEXT_SAC_DURATION,NEXT_SAC_ANGLE,NEXT_SAC_AVG_VELOCITY,DATETIME,WEBSITE_ID,CONDITION,OFFSET,Y_OFFSET
0,1,RIGHT,775.5,564.5,176,53,882.6,497.0,3.75,RIGHT,114.0,34.88,32.91,2018-04-03 17:21:14.165,13,5,0,564.5
1,1,RIGHT,890.3,487.6,343,179,798.1,547.4,3.16,LEFT,37.0,-146.84,85.51,2018-04-03 17:21:14.332,13,5,0,487.6
2,1,RIGHT,797.1,559.2,559,151,814.5,656.3,2.76,DOWN,27.0,-79.39,102.09,2018-04-03 17:21:14.548,13,5,0,559.2
3,1,RIGHT,820.4,650.1,737,215,847.7,196.7,12.64,UP,52.0,86.35,243.03,2018-04-03 17:21:14.726,13,5,0,650.1
4,1,RIGHT,845.1,194.7,1004,205,933.1,163.2,2.73,RIGHT,47.0,17.47,58.11,2018-04-03 17:21:14.993,13,5,0,194.7
